In [11]:
from copy import deepcopy
import json
from typing import List, Optional, Tuple, Type, TypeVar
from tqdm import tqdm
import time
from pydantic.dataclasses import dataclass

In [12]:
model="llama2-70b"

In [13]:
# get a token: https://replicate.com/account
from getpass import getpass
import os

REPLICATE_API_TOKEN = getpass()
os.environ["REPLICATE_API_TOKEN"] = "r8_IwYEZpurE3QujsWwrE2t7EaLw5cRqo81AjBOn"

In [14]:
from groq import Groq
client = Groq(
    api_key="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"
)

In [15]:
def get_closedbook_qa_prompt(question: str):
    with open("./prompting/closedbook_qa.prompt") as f:
        prompt_template = f.read().rstrip("\n")

    return prompt_template.format(question=question)

In [17]:
def get_closedbook_responses(inp, out):
    prompts = []
    responses = []
    correct_responses = []
    
    with open(inp) as fin:
        for line in tqdm(fin):
            input_example = json.loads(line)
            
            # Getting question and correct answer
            question = input_example["question"]
            correct_answer = input_example["answers"]
            
            qa_closedbook_prompt = get_closedbook_qa_prompt(question)
            
            prompts.append(qa_closedbook_prompt)
            correct_responses.append(correct_answer)
            
            chat_completion = client.chat.completions.create(
                messages=[
                    {
                        "role": "user",
                        "content": qa_closedbook_prompt,
                    }
                ],
                model="llama2-70b-4096",  
            )
            response = chat_completion.choices[0].message.content

            responses.append(response)
            time.sleep(2) 
             
            
    with open(out, "w") as f:
        for prompt, response, correct_answer in zip(prompts, responses, correct_responses):
            output = {}

            output["model_prompt"] = prompt
            output["model_answer"] = response
            output["model"] = model
            output["correct_answer"] = correct_answer

            f.write(json.dumps(output) + "\n")

In [18]:
input_path = "./qa_data/nq-open-oracle.json"
output_path = "./responses/llama2_qa/llama2_70b_closedbook_responses.jsonl"

In [19]:
get_closedbook_responses(input_path, output_path)

0it [00:00, ?it/s]

2655it [2:16:12,  3.08s/it]
